# 3_training_megatron-lm

- https://github.com/NVIDIA/Megatron-LM 에서 파일을 다운로드 받은 후 3_training_megatron-lm 폴더의 코드를 덮어쓰기하여 학습을 하시면 됩니다.
- 시작 전 docker 폴더에서 Dockerfile을 이용하여 custom_docker를 만들어야 합니다.

In [1]:
install_needed = True
# install_needed = False

In [2]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

sudo curl -L "https://github.com/docker/compose/releases/download/v2.7.0/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
sudo chmod +x /usr/local/bin/docker-compose

Already revised


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 24.5M    0  1362    0     0  10034      0  0:42:50 --:--:--  0:42:50 10034
curl: (23) Failure writing output to destination


In [3]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install --upgrade pip --quiet
    !{sys.executable} -m pip install -U sagemaker --quiet
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel


## Building and registering the container

In [1]:
%%sh

# The name of our algorithm
algorithm_name=pytorch-training
image_tag=2.0.0-gpu-py310-apex

cd '3_training_megatron-lm/docker'

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:${image_tag}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -f Dockerfile -t ${fullname} .

docker push ${fullname}

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded
Sending build context to Docker daemon   7.68kB
[WARNING]: Empty continuation line found in:
    RUN git clone https://github.com/NVIDIA/apex   && cd apex   && git checkout 27de66c   && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./   && cd .. RUN pip install regex
[WARNING]: Empty continuation lines will become errors in a future release.
Step 1/3 : FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:2.0.0-gpu-py310
 ---> 2cf7a91d14ca
Step 2/3 : RUN git clone https://github.com/NVIDIA/apex   && cd apex   && git checkout 27de66c   && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./   && cd .. RUN pip install regex
 ---> Using cache
 ---> 2c3787173d6a
Step 3/3 : WORKDIR /
 ---> Using cache
 ---> ab63d7aec05a
Successfully built ab63d7aec05a
Successfully tagged 322537213286.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:2.0.0-gpu-py310-apex
The push refers to repository [32

## Import Setting

In [62]:
import sagemaker
from pathlib import Path
from time import strftime

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/megatron-lm'

role = sagemaker.get_execution_role()

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [63]:
sagemaker.__version__

'2.174.0'

In [181]:
# Configure FSx Input for your SageMaker Training job
from sagemaker.inputs import FileSystemInput

file_system_directory_path= '/dataset' #
file_system_id='fs-XXXXXXXXXXXXX'         # 
file_system_access_mode='ro'
file_system_type='EFS'
train_fs=FileSystemInput(file_system_id=file_system_id,
                         file_system_type=file_system_type,
                         directory_path=file_system_directory_path,
                         file_system_access_mode=file_system_access_mode)

### Model training with Distributed Data Parallel


The training script provides the code you need for distributed data parallel (DDP) training. The training script is very similar to a PyTorch training script you might run outside of SageMaker.

In the following code block, you can update the estimator function to use a different instance type, instance count, and distrubtion strategy. You're also passing in the training script you reviewed in the previous cell.

In [182]:
hyperparameters = {
    'num-layers': 12,
    'hidden-size': 768,
    'num-attention-heads': 12,
    'seq-length': 1024,
    'max-position-embeddings': 1024,
    'micro-batch-size': 12,
    'global-batch-size': 192,
    'lr': 0.0005,
    # 'train-iters': 150000,
    'train-iters': 4000,
    'lr-decay-iters': 150000,
    'lr-decay-style': 'cosine',
    'lr-warmup-iters': 2000,
    'weight-decay':  .1,
    'adam-beta2':  .999,
    'fp16' : 'true',
    'log-interval': 10,
    'save-interval': 2000,
    'eval-interval': 200,
    'eval-iters': 10,
    'data-path':'/opt/ml/input/data/dataset/codeparrot_content_document',
    'vocab-file':'/opt/ml/input/data/dataset/gpt2-vocab.json',
    'merge-file':'/opt/ml/input/data/dataset/gpt2-merges.txt',
    'save' : '/opt/ml/model/',
    'tensor-model-parallel-size' : 4,
    'pipeline-model-parallel-size' : 1
}

In [183]:
distribution = {}
distribution["mpi"]={"enabled": True}

In [184]:
# instance_type = 'ml.p3.16xlarge'  # 'ml.p3.16xlarge', 'ml.p3dn.24xlarge', 'ml.p4d.24xlarge', 'local_gpu'
instance_type = 'ml.p4d.24xlarge'
# instance_type = 'local_gpu'
instance_count = 2
max_run = 2*60*60

In [185]:
if instance_type =='local_gpu':
    from sagemaker.local import LocalSession

    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    s3_data_path = f'file://{Path.cwd()}/codeparrot'
    model_weight = ''
else:
    sagemaker_session = sagemaker.Session()
    s3_data_path = "s3://bucket-name-XXXXXXXX/megatron-lm/codeparrot/"
    s3_data_path = train_fs
s3_data_path

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


## `Pytorch` estimator를 이용한 training job 생성하기


<p><strong><code>sagemaker.pytorch.PyTorch</code></strong> estimator는 처음 실행하는 스크립트 위치와 다양한 연계 코드들이 위치한 디렉토리 정보를 찾아서 스크립트를 S3에 upload하고 SageMaker의 training job을 수행하게 됩니다. training job은 학습을 수행한 단위입니다. 학습을 1번 돌리면 training job이 1개 생성됩니다. 몇 가지 중요 파라미터를 아래와 같이 설명드립니다. </p>

- **entry_point** : 학습을 처음 실행하는 Python 소스 파일의 절대 또는 상대 경로이며, source_dir이 지정된 경우 entry_point는 source_dir 내 파일이 됩니다.
- **source_dir** : 학습에 연계되는 다양한 소스코드 파일이 들어 있는 디렉토리 위치이며, 절대, 상대 경로 또는 S3 URI가 모두 가능하며,source_dir이 S3 URI 인 경우 tar.gz 파일이 됩니다.
- **role** : Amazon SageMaker가 사용자를 대신해 작업(예: S3 버킷에서 모델 결과물이라고 하는 훈련 결과 읽기 및 Amazon S3에 훈련 결과 쓰기)을 수행하는 AWS Identity and Access Management(IAM) 역할입니다.
- **train_instance_count** : 학습을 수행하는 instance 개수를 정의할 수 있습니다.
- **train_instance_type** : 학습을 수행하는 instance 타입을 정의할 수 있습니다.
- **train_volume_size** : 학습 인스턴스에 연결할 Amazon Elastic Block Store(Amazon EBS) 스토리지 볼륨의 크기(GB)입니다. File 모드를 사용할 경우 이 값이 훈련 데이터를 충분히 저장할 수 있는 크기여야 합니다(File 모드가 기본값)
- **train_max_run** : 최대 학습 시간을 설정할 수 있으며, 이 시간이 지나면 Amazon SageMaker는 현재 상태에 관계없이 작업을 종료합니다. (기본값 : 24 * 60 * 60)
- **framework_version** : 학습에 사용될 특정 Pytorch 버전을 정의할 수 있습니다.
- **py_version** : 컨테이너 환경이 python3일 경우 py3, python2일 경우 py2로 설정하면 됩니다. python2는 지원이 중단되었지만 기존 python2로 구성된 파일들을 지원하기 위해 현재 계속 사용할 수 있습니다. 없을 경우에는 기본적으로 py3 입니다.
- **hyperparameters** : 학습에 사용할 하이퍼 파라미터를 정의할 수 있으며, 정의된 하이퍼 파라미터 값들은 모두 학습 컨테이너로 전송이 됩니다.
- **distribution** : 분산과 관련된 값들을 학습 컨테이너로 전송합니다.

<p> 추가적으로 분산/ 멀티 GPU 학습도 가능합니다. SageMaker는 <strong><a href="https://github.com/horovod/horovod" target="_blank" class ='btn-default'>Horovod</a></strong>에 최적화된 환경을 제공하고 있으며, Pytorch의 경우 1.5.0부터 기본 docker에서 apex를 지원합니다.</p>


In [186]:
image_uri = f'{accound_id}.dkr.ecr.{region_name}.amazonaws.com/pytorch-training:2.0.0-gpu-py310-apex'

In [187]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
                    # entry_point='train_adv.py',
                    entry_point='pretrain_gpt.py',
                    source_dir=f'{Path.cwd()}/3_training_megatron-lm',
                    role=role,
                    image_uri=image_uri,
                    framework_version='1.13.1',
                    py_version='py39',
                    instance_count=instance_count,
                    instance_type=instance_type,
                    distribution=distribution,
                    # metric_definitions=metric_definitions,
                    disable_profiler=True,
                    debugger_hook_config=False,
                    max_run=max_run,
                    hyperparameters=hyperparameters,
                    sagemaker_session=sagemaker_session,
                   )

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [188]:
!sudo rm -rf code/core.*

In [189]:
current_time = strftime("%m%d-%H%M%s")
i_type = instance_type.replace('.','-')
job_name = f'megatron-lm-{i_type}-{current_time}'

estimator.fit(
    inputs={'dataset': s3_data_path, 'model_weight': model_weight}, 
    job_name=job_name,
    wait=False
)

INFO:sagemaker:Creating training-job with name: megatron-lm-ml-p4d-24xlarge-0805-12361691239004


Using provided s3_resource


In [190]:
sagemaker_session = sagemaker.Session()
sagemaker_session.logs_for_job(job_name=job_name, wait=True)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


2023-08-05 12:36:46 Starting - Starting the training job......
2023-08-05 12:37:31 Starting - Preparing the instances for training........................
2023-08-05 12:41:35 Downloading - Downloading input data
2023-08-05 12:41:35 Training - Downloading the training image..............................
2023-08-05 12:46:21 Training - Training image download completed. Training in progress.......bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-08-05 12:47:35,821 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-08-05 12:47:35,877 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-08-05 12:47:35,884 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-08-05 12:47:35,886 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-08-05 12:47:37,361 sagemaker-training-toolki

KeyboardInterrupt: 